In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, callback
import mlflow
from utils.metrics import Metrics_df,print_metrics, precision, recall, f1


In [3]:


import pandas as pd
from sklearn.metrics import roc_auc_score
from Config import config
import warnings
warnings.filterwarnings("ignore")
import optuna  # pip install optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc
import pickle
import tempfile

In [4]:
from lightgbm.basic import LightGBMError
from lightgbm import LGBMClassifier, early_stopping, log_evaluation, Booster
from sklearn.metrics import roc_auc_score
import os
from catboost import CatBoostClassifier


# Loading data and group split

In [ ]:
# df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data.csv")
# df_pairs = df_pairs.sample(frac=1).reset_index(drop=True)
# kf = GroupKFold(n_splits=5)
# for i, (trn_idx, val_idx) in enumerate(kf.split(df_pairs, df_pairs["placeId1"], df_pairs["placeId1"])):
#     df_pairs.loc[val_idx, "fold"] = i
# df_pairs["fold"].value_counts()
# df_pairs.to_csv("/workspace/Training_data/training/Full_training_data_fold.csv",index=None)

# Loading data

In [5]:
df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data_fold_v4_without_categories.csv")

In [6]:
features = ['similarity', 'haversine', 'name_davies',
       'name_leven', 'name_dice', 'name_jaro', 'name_set_ratio',
       'street_davies', 'street_leven', 'street_jaro', 'email_davies',
       'email_leven', 'email_jaro', 'url_davies', 'url_leven', 'url_jaro',
       'brands_davies', 'brand_leven', 'brand_jaro', 'phone_lcs',
       'subcat_WRatio', 'subcat_ratio', 'subcat_token_set_ratio',
       'Is_direction_match_0', 'Is_direction_match_1', 'Is_direction_match_2',
       'Is_house_match_0', 'Is_house_match_1', 'Is_house_match_2',
       'Is_category_match_0', 'Is_category_match_1', 'Is_subcategory_match_0',
       'Is_subcategory_match_1', 'Is_subcategory_match_2', 'Is_brand_match_0',
       'Is_brand_match_1', 'Is_brand_match_2', 'Is_brand_match_3',
       'Is_related_cat_0', 'Is_related_cat_1', 'Is_name_number_match_0',
       'Is_name_number_match_1', 'Is_name_number_match_2',
       'Is_name_number_match_3', 'is_phone_match_1', 'is_phone_match_2',
       'is_phone_match_3', 'is_phone_match_4', 'Is_email_match_0',
       'Is_email_match_1', 'Is_email_match_2', 'Is_url_match_0',
       'Is_url_match_1', 'Is_url_match_2']

In [7]:
def objective(trial, df_pairs):
    catboost_params = dict(
        depth=trial.suggest_int("depth", 5, 15),
        border_count=trial.suggest_int("border_count", 16, 256),
        min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 100, 10000),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-3, 1, log=True)
    )
    
    cv_scores = np.empty(5)
    cv_scores_train = np.empty(5)
    for fold in [0, 1, 2, 3, 4]:
        X_train = df_pairs[df_pairs["fold"] != fold]
        X_test = df_pairs[df_pairs["fold"] == fold]

        model = CatBoostClassifier(
            **catboost_params,
            learning_rate=0.1,
            n_estimators=10000,
            objective="Logloss",
            eval_metric="AUC",
            gpu_ram_part=0.90,
            task_type="GPU",
            use_best_model = True
        )
        
        model.fit(X_train[features],
                  X_train["duplicate_flag"],verbose=100,
                  eval_set=[(X_test[features], X_test["duplicate_flag"])],
                  early_stopping_rounds=40)

        preds = model.predict_proba(X_test[features])[:, 1]
        preds_train = model.predict_proba(X_train[features])[:, 1]
        cv_scores[fold] = roc_auc_score(X_test["duplicate_flag"], preds)
        cv_scores_train[fold] = roc_auc_score(X_train["duplicate_flag"],
                                              preds_train)
    print(f"train auc score = {np.mean(cv_scores_train)}")
    print(f"test auc score = {np.mean(cv_scores)}")
    mlflow.log_metric('train_auc', np.mean(cv_scores_train))
    mlflow.log_metric('test_auc', np.mean(cv_scores))
    return np.mean(cv_scores)

In [8]:
date_log = datetime.now().strftime("%Y_%m_%d_%H_%M")
study = optuna.create_study(direction="maximize", study_name=f"parameter_tuning_{config.country}_{date_log}")
func = lambda trial: objective(trial, df_pairs)
study.optimize(func, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2023-01-13 12:20:12,841] A new study created in memory with name: parameter_tuning_NZL_2023_01_13_12_20
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9964053	best: 0.9964053 (0)	total: 135ms	remaining: 22m 25s
100:	test: 0.9998855	best: 0.9998855 (100)	total: 12.4s	remaining: 20m 13s
200:	test: 0.9999117	best: 0.9999117 (200)	total: 24.2s	remaining: 19m 41s
bestTest = 0.9999126196
bestIteration = 218
Shrink model to first 219 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9968840	best: 0.9968840 (0)	total: 120ms	remaining: 19m 57s
100:	test: 0.9999019	best: 0.9999019 (100)	total: 12.2s	remaining: 19m 52s
200:	test: 0.9999133	best: 0.9999137 (195)	total: 23.9s	remaining: 19m 23s
300:	test: 0.9999220	best: 0.9999223 (293)	total: 36.1s	remaining: 19m 23s
400:	test: 0.9999254	best: 0.9999254 (400)	total: 48.6s	remaining: 19m 22s
500:	test: 0.9999298	best: 0.9999300 (491)	total: 1m	remaining: 19m 14s
bestTest = 0.9999315739
bestIteration = 526
Shrink model to first 527 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.9987736	best: 0.9987736 (0)	total: 88.1ms	remaining: 14m 40s
100:	test: 0.9999188	best: 0.9999188 (100)	total: 8.24s	remaining: 13m 28s
200:	test: 0.9999288	best: 0.9999288 (200)	total: 16.3s	remaining: 13m 15s
300:	test: 0.9999343	best: 0.9999343 (299)	total: 24.4s	remaining: 13m 5s
400:	test: 0.9999371	best: 0.9999371 (400)	total: 32.4s	remaining: 12m 56s
bestTest = 0.9999371767
bestIteration = 408
Shrink model to first 409 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9988322	best: 0.9988322 (0)	total: 217ms	remaining: 36m 5s
100:	test: 0.9999237	best: 0.9999237 (100)	total: 8.53s	remaining: 13m 55s
200:	test: 0.9999337	best: 0.9999341 (190)	total: 19.4s	remaining: 15m 47s
300:	test: 0.9999378	best: 0.9999378 (300)	total: 27.3s	remaining: 14m 39s
400:	test: 0.9999404	best: 0.9999407 (397)	total: 35.4s	remaining: 14m 6s
bestTest = 0.9999406934
bestIteration = 397
Shrink model to first 398 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9988567	best: 0.9988567 (0)	total: 85.3ms	remaining: 14m 12s


In [12]:
study.trials_dataframe().to_csv("/workspace/catboost_trials.csv",index=None)